# Importamos librerias

In [1]:
import pandas as pd
import numpy as np
import os
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [2]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

Connected


# Segmentos

In [3]:
all_files = os.listdir()

excel_files = [file for file in all_files if 'Ofertata' in file and file.endswith('.xlsx')]

df = pd.DataFrame()
for file in excel_files:
    df_aux = pd.read_excel(file)
    df_aux['SEGMENTO'] = file.split(' - ')[0].split(', ')[1]
    df = pd.concat([df, df_aux])

df.head(2)

,SOCI_SOCI_ID,SUBCLASE,CORREO,CELULAR,SUB_NAME,SEGMENTO
0,738541,2700020001,patovivi803@gmail.com,95848778.0,A BASE DE TOMATE,S3
1,725504,2700020001,NaN,99351289.0,A BASE DE TOMATE,S3


In [4]:
df['SEGMENTO'].value_counts()

S3    164395
S5    150933
Name: SEGMENTO, dtype: int64

# Segmento 5

In [5]:
# Clientes que en el ultimo mes no vengan a tata y tengan Score 5 en la subclase

In [48]:
segm_5 = df[df['SEGMENTO'] == 'S5']
segm_5['K'] = segm_5['SOCI_SOCI_ID'].astype(str) + segm_5['SUBCLASE'].astype(str)

In [7]:
segm_5['SUB_NAME'].unique()

array(['A BASE DE TOMATE', 'ATUN', 'CERVEZAS IMPORTADAS', 'CUADRADA',
       'DURAZNOS REGULARES', 'YERBA TRADICIONAL'], dtype=object)

## Impactados

### A. Vinieron a TATA

In [8]:
query = '''
SELECT
    DISTINCT FFM.SOCI_SOCI_ID
FROM
    MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
WHERE
    FFM.TIEM_DIA_ID > '2024-01-24'
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
'''

cursor.execute(query)
volvieron_general = cursor.fetch_pandas_all()
volvieron_general = volvieron_general[volvieron_general['SOCI_SOCI_ID'].isin(segm_5['SOCI_SOCI_ID'])]
print(volvieron_general['SOCI_SOCI_ID'].nunique())
volvieron_general.head(2)

21891


,SOCI_SOCI_ID
30,152371
106,324014


### B. Vinieron en 2024 y consumieron alguna de las SUBCLASE

In [9]:
query = '''
SELECT
    FFM.TICKET,
    FFM.SOCI_SOCI_ID,
    SUB.SUB_NAME
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    SUB.SUB_NAME IN {subclases_snow}
    AND FFM.TIEM_DIA_ID > '2024-01-24'
'''

cursor.execute(query.format(subclases_snow = tuple(segm_5['SUB_NAME'].unique())))
volvieron_subclase = cursor.fetch_pandas_all()

volvieron_subclase.head(2)

,TICKET,SOCI_SOCI_ID,SUB_NAME
0,2024012830383345,415855,ATUN
1,202401283033656,2166376,ATUN


In [10]:
print(volvieron_subclase.shape[0])
volvieron_subclase_2 = volvieron_subclase[(volvieron_subclase['SOCI_SOCI_ID'].isin(segm_5['SOCI_SOCI_ID'])) & (volvieron_subclase['SUB_NAME'].isin(segm_5['SUB_NAME']))]
print(volvieron_subclase_2.shape[0])
print(volvieron_subclase_2['SOCI_SOCI_ID'].nunique())

247454
11001
5275


In [11]:
# Compraron
resumen_vinieron_2024_2 = volvieron_subclase_2.groupby('SUB_NAME')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_vinieron_2024_2.rename({'SOCI_SOCI_ID':'COMPRARON'}, axis = 1, inplace = True)

# Impactados
impactados_segm_5 = segm_5.groupby('SUB_NAME')['SOCI_SOCI_ID'].nunique().reset_index()
impactados_segm_5.rename({'SOCI_SOCI_ID':'IMPACTADOS'}, axis = 1, inplace = True)

# Consolido
resumen_vinieron_2024_2 = resumen_vinieron_2024_2.merge(impactados_segm_5, on = 'SUB_NAME', how = 'inner')
resumen_vinieron_2024_2['RATIO'] = (resumen_vinieron_2024_2['COMPRARON'] / resumen_vinieron_2024_2['IMPACTADOS'] * 100).astype(int)
resumen_vinieron_2024_2.sort_values(by = 'RATIO', ascending = False)

,SUB_NAME,COMPRARON,IMPACTADOS,RATIO
1,ATUN,2290,22103,10
5,YERBA TRADICIONAL,1943,24599,7
2,CERVEZAS IMPORTADAS,1103,26027,4
4,DURAZNOS REGULARES,637,23286,2
0,A BASE DE TOMATE,318,23956,1
3,CUADRADA,290,30962,0


### Conclusiones Impactados

In [12]:
a = segm_5['SOCI_SOCI_ID'].nunique()
b = int(volvieron_general['SOCI_SOCI_ID'].nunique() / segm_5['SOCI_SOCI_ID'].nunique() * 100)
c = int(volvieron_subclase_2['SOCI_SOCI_ID'].nunique() / segm_5['SOCI_SOCI_ID'].nunique() * 100)

print("Los segmentos de Ofertata 4 fueron generados el 24/01/2024")
print("")
print("---- Segmento 5")
print("")
print(f"Comprende clientes que en los ultimos 30 dias no vengan a TATA (25/12/2023 - 24/01/2024) y tengan Score 5 en la subclase")
print(f"De los {a} clientes que cumplen dicho criterio, un {b}% volvieron a TATA luego del Ofertata 4 (25/01/2024 en adelante)")
print("")
print(f"Asimismo, de los {a} clientes, un {c}% volvieron a TATA luego del Ofertata 4 (25/01/2024 en adelante) Y compraron la subclase que se les promociono, siendo ATUN y YERBA TRADICIONAL las mejores subclases")

Los segmentos de Ofertata 4 fueron generados el 24/01/2024

---- Segmento 5

Comprende clientes que en los ultimos 30 dias no vengan a TATA (25/12/2023 - 24/01/2024) y tengan Score 5 en la subclase
De los 150933 clientes que cumplen dicho criterio, un 14% volvieron a TATA luego del Ofertata 4 (25/01/2024 en adelante)

Asimismo, de los 150933 clientes, un 3% volvieron a TATA luego del Ofertata 4 (25/01/2024 en adelante) Y compraron la subclase que se les promociono, siendo ATUN y YERBA TRADICIONAL las mejores subclases


## Venta y GB1

In [50]:
query = '''
SELECT
    FFM.SOCI_SOCI_ID,
    FFM.SOCI_SOCI_ID || SUB.SUBCLASE AS K,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) - SUM(FV.VNTA_UNIDADES * FV.VNTA_COSTO_PROM_POND) AS GB1
FROM
    MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    FFM.TIEM_DIA_ID BETWEEN '2024-01-24' AND '2024-01-31'
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND SUB.SUB_NAME IN {subclases_query}
GROUP BY
    ALL
'''

cursor.execute(query.format(
    subclases_query = tuple(segm_5['SUB_NAME'].unique())
    ))
ventas_segm_5 = cursor.fetch_pandas_all()

ventas_segm_5 = ventas_segm_5[ventas_segm_5['K'].isin(segm_5['K'])]

print(f"Ventas en Miles: {int(round(ventas_segm_5['VENTAS'].sum() / 1000, 0))}")
print(f"GB1 en Miles: {int(round(ventas_segm_5['GB1'].sum() / 1000, 0))}")

Ventas en Miles: 140
GB1 en Miles: 26


# Segmento 3

In [34]:
# Para cada categoria (subclase) son los clientes activos que no compran la subclase y tienen score 5

In [35]:
segm_3 = df[df['SEGMENTO'] == 'S3']
segm_3['K'] = segm_3['SOCI_SOCI_ID'].astype(str) + segm_3['SUBCLASE'].astype(str)

In [36]:
segm_3['SUB_NAME'].unique()

array(['A BASE DE TOMATE', 'ATUN', 'CERVEZAS IMPORTADAS', 'CUADRADA',
       'DURAZNOS REGULARES', 'YERBA TRADICIONAL'], dtype=object)

In [37]:
query = '''
SELECT
    FFM.TICKET,
    FFM.SOCI_SOCI_ID,
    SUB.SUB_NAME
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    SUB.SUB_NAME IN {subclases_snow}
    AND FFM.TIEM_DIA_ID > '2024-01-24'
'''

cursor.execute(query.format(subclases_snow = tuple(segm_3['SUB_NAME'].unique())))
volvieron_subclase_segm_3 = cursor.fetch_pandas_all()

volvieron_subclase_segm_3.head(2)

,TICKET,SOCI_SOCI_ID,SUB_NAME
0,2024012830383345,415855,ATUN
1,202401283033656,2166376,ATUN


In [38]:
print(volvieron_subclase_segm_3.shape[0])
volvieron_subclase_segm_3_version_2 = volvieron_subclase_segm_3[(volvieron_subclase_segm_3['SOCI_SOCI_ID'].isin(segm_3['SOCI_SOCI_ID'])) & (volvieron_subclase_segm_3['SUB_NAME'].isin(segm_3['SUB_NAME']))]
print(volvieron_subclase_segm_3_version_2.shape[0])
print(volvieron_subclase_segm_3_version_2['SOCI_SOCI_ID'].nunique())

247454
121836
46443


In [39]:
# Compraron
volvieron_subclase_segm_3_version_2_resumen = volvieron_subclase_segm_3_version_2.groupby('SUB_NAME')['SOCI_SOCI_ID'].nunique().reset_index()
volvieron_subclase_segm_3_version_2_resumen.rename({'SOCI_SOCI_ID':'COMPRARON'}, axis = 1, inplace = True)

# Impactados
impactados_segm_3 = segm_3.groupby('SUB_NAME')['SOCI_SOCI_ID'].nunique().reset_index()
impactados_segm_3.rename({'SOCI_SOCI_ID':'IMPACTADOS'}, axis = 1, inplace = True)

# Consolido
volvieron_subclase_segm_3_version_2_resumen = volvieron_subclase_segm_3_version_2_resumen.merge(impactados_segm_3, on = 'SUB_NAME', how = 'inner')
volvieron_subclase_segm_3_version_2_resumen['RATIO'] = (volvieron_subclase_segm_3_version_2_resumen['COMPRARON'] / volvieron_subclase_segm_3_version_2_resumen['IMPACTADOS'] * 100).astype(int)
volvieron_subclase_segm_3_version_2_resumen.sort_values(by = 'RATIO', ascending = False)

,SUB_NAME,COMPRARON,IMPACTADOS,RATIO
1,ATUN,19701,19974,98
5,YERBA TRADICIONAL,18758,20803,90
2,CERVEZAS IMPORTADAS,11496,28013,41
4,DURAZNOS REGULARES,6392,28034,22
0,A BASE DE TOMATE,3964,32388,12
3,CUADRADA,3701,35183,10


In [40]:
a = segm_3['SOCI_SOCI_ID'].nunique()
b = int(volvieron_subclase_segm_3_version_2['SOCI_SOCI_ID'].nunique() / segm_3['SOCI_SOCI_ID'].nunique() * 100)

print("---- Segmento 3")
print("")
print(f"El Segmento 3 comprende a los clientes activos que no compran la subclase y tienen score 5 en la misma")
print("")
print(f"De los {a} clientes que cumplen dicho criterio, un {b}% volvieron a TATA luego del Ofertata 4 (25/01/2024 en adelante) Y compraron la subclase que se les promociono, siendo ATUN y YERBA TRADICIONAL las mejores subclases")

---- Segmento 3

El Segmento 3 comprende a los clientes activos que no compran la subclase y tienen score 5 en la misma

De los 164395 clientes que cumplen dicho criterio, un 28% volvieron a TATA luego del Ofertata 4 (25/01/2024 en adelante) Y compraron la subclase que se les promociono, siendo ATUN y YERBA TRADICIONAL las mejores subclases


## Venta y GB1

In [47]:
query = '''
SELECT
    FFM.SOCI_SOCI_ID,
    FFM.SOCI_SOCI_ID || SUB.SUBCLASE AS K,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) - SUM(FV.VNTA_UNIDADES * FV.VNTA_COSTO_PROM_POND) AS GB1
FROM
    MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    FFM.TIEM_DIA_ID BETWEEN '2024-01-24' AND '2024-01-31'
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND SUB.SUB_NAME IN {subclases_query}
GROUP BY
    ALL
'''

cursor.execute(query.format(
    subclases_query = tuple(segm_3['SUB_NAME'].unique())
    ))
ventas_segm_3 = cursor.fetch_pandas_all()

ventas_segm_3 = ventas_segm_3[ventas_segm_3['K'].isin(segm_3['K'])]

print(f"Ventas en Miles: {int(round(ventas_segm_3['VENTAS'].sum() / 1000, 0))}")
print(f"GB1 en Miles: {int(round(ventas_segm_3['GB1'].sum() / 1000, 0))}")

Ventas en Miles: 482
GB1 en Miles: 69


# Fin